In [1]:
# Install
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, ElectraForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np
import evaluate
import torch
import pandas as pd

# Load the dataset
df = pd.read_excel('/content/final_labels_SG2.xlsx')

df = df[['text', 'label_bias']]
df.columns = ['text', 'label']
# Print unique values in the 'label' column to identify the issue
print(df['label'].unique())
# Replace all unique values in 'label' column with numerical values
df['label'] = df['label'].replace({'Non-biased': 0, 'Biased': 1})
dataset = Dataset.from_pandas(df)

['Non-biased' 'Biased']


<ipython-input-2-8965b3461e4f>:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'Non-biased': 0, 'Biased': 1})


In [3]:
dataset = dataset.train_test_split(test_size=0.2, seed=42) # Split into train and test (80% train, 20% test)
train_valid_dataset = dataset['train'].train_test_split(test_size=0.1, seed=42) # Split train into train and validation (10% of train, which is 8% of the original dataset)

# Update the dataset dictionary with train, validation, and test sets
dataset = DatasetDict({
    'train': train_valid_dataset['train'],
    'validation': train_valid_dataset['test'],
    'test': dataset['test']
})

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
# Split the dataset
train_dataset = dataset["train"]
val_dataset = dataset["test"]

# Define the model and tokenizer
model_name = "google/electra-base-discriminator"  # Using DistilBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2) # Assuming binary classification

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Tokenize the Dataset
def tokenize_function(examples):
    # Filter out empty or None values before tokenization
    examples['text'] = [text for text in examples['text'] if text is not None and text.strip()]

    # If 'text' is still empty after filtering, assign an empty string
    examples['text'] = [text if text else "" for text in examples['text']]

    # Handle potential None values in 'label'
    examples['label'] = [label if label is not None else 0 for label in examples['label']]

    # Add this debugging line to inspect the input to the tokenizer
    #print("Examples being tokenized:", examples)

    # Adjust max_length and add padding to handle longer sequences
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128, return_tensors="pt")

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2645 [00:00<?, ? examples/s]

Map:   0%|          | 0/735 [00:00<?, ? examples/s]

In [7]:
# Define Metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)

    # Compute accuracy
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    # Compute F1 score
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Compute recall
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Compute precision
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')

    # Return all metrics
    return {**accuracy, **f1, **recall, **precision}


In [8]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=2,
    weight_decay=0.01,
    gradient_accumulation_steps=8, # Accumulate gradients
    fp16=True,  # Enable mixed precision training if your GPU supports it
    gradient_checkpointing=True # Enable gradient checkpointing
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the Model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-1d200b9bffdf>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
0,0.405500,0.474118,0.798639,0.795558,0.798639,0.813361
1,0.343300,0.458921,0.813605,0.813592,0.813605,0.814667


TrainOutput(global_step=164, training_loss=0.4262014627456665, metrics={'train_runtime': 119.6409, 'train_samples_per_second': 44.216, 'train_steps_per_second': 1.371, 'total_flos': 345201704632320.0, 'train_loss': 0.4262014627456665, 'epoch': 1.9833836858006042})

In [9]:
trainer.evaluate()

{'eval_loss': 0.4589209258556366,
 'eval_accuracy': 0.8136054421768707,
 'eval_f1': 0.8135923308887179,
 'eval_recall': 0.8136054421768707,
 'eval_precision': 0.8146674542833006,
 'eval_runtime': 2.8564,
 'eval_samples_per_second': 257.319,
 'eval_steps_per_second': 64.417,
 'epoch': 1.9833836858006042}